In [ ]:
import numpy as np
import pandas as pd
from fbprophet import Prophet

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_train = pd.read_csv("/kaggle/input/sz-water/train.csv")
df_test = pd.read_csv("/kaggle/input/sz-water/test.csv")
df_submission = pd.read_csv("/kaggle/input/sz-water/submission.csv")

df_train_a = df_train[["日期","A厂"]].rename(columns={"日期":"ds","A厂":"y"})
df_train_b = df_train[["日期","B厂"]].rename(columns={"日期":"ds","B厂":"y"})

mean_a,std_a = df_train_a["y"].mean(),df_train_a["y"].std()
mean_b,std_b = df_train_b["y"].mean(),df_train_b["y"].std()

df_train_a["y"] = (df_train_a["y"] - mean_a) / std_a
df_train_b["y"] = (df_train_b["y"] - mean_b) / std_b

df_test_a = df_test[["日期","A厂"]].rename(columns={"日期":"ds","A厂":"y"})
df_test_b = df_test[["日期","B厂"]].rename(columns={"日期":"ds","B厂":"y"})

In [ ]:
df_train_a

In [ ]:
df_test_a

In [ ]:
df_submission

In [ ]:
print(df_train_a["y"].skew(),df_train_a["y"].kurt())
df_train_a[["ds","y"]].plot()

In [ ]:
# 2018-01-01 2021-03-31
holidays_2018 = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2018-01-01',
                        '2018-02-15','2018-02-16','2018-02-17','2018-02-18','2018-02-19','2018-02-20','2018-02-21', 
                        '2018-04-05','2018-04-06','2018-04-07',
                        '2018-04-29', '2018-04-30', '2018-05-01',
                        '2018-06-16', '2018-06-17', '2018-06-18',
                        '2018-09-22', '2018-09-23', '2018-09-24',
                        '2018-10-01', '2018-10-02', '2018-10-03','2018-10-04', '2018-10-05', '2018-10-06', '2018-10-07',
                        '2018-12-30', '2018-12-31']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays_2019 = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2019-01-01',
                        '2019-02-04', '2019-02-05', '2019-02-06','2019-02-07','2019-02-08','2019-02-09','2019-02-10', 
                        '2019-04-05', '2019-04-06', '2019-04-07',
                        '2019-05-01', '2019-05-02', '2019-05-03', '2019-05-04',
                        '2019-06-07', '2019-06-08', '2019-06-09',
                        '2019-09-13', '2019-09-14', '2019-09-15',
                        '2019-10-01', '2019-10-02', '2019-10-03','2019-10-04', '2019-10-05', '2019-10-06','2019-10-07']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays_2020 = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2020-01-01', '2020-01-02', '2020-01-03',
                        '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02',  
                        '2020-04-04', '2020-04-05', '2020-04-06',
                        '2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05',
                        '2020-06-25', '2020-06-26', '2020-06-27',
                        '2020-10-01', '2020-10-02', '2020-10-03','2020-10-04', '2020-10-05', '2020-10-06','2020-10-07','2020-10-08']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays_2021 = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2021-01-01', '2021-01-02', '2021-01-03',
                        '2021-02-11', '2021-02-12', '2021-02-13', '2021-02-14', '2021-02-15', '2018-02-16', '2018-02-17']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays = pd.concat((holidays_2018, holidays_2019, holidays_2020, holidays_2021))

In [ ]:
ph_a = Prophet(holidays=holidays, holidays_prior_scale=10.0)
ph_a.fit(df_train_a)
forecast_a = ph_a.predict(pd.concat([df_train_a[['ds']],df_test_a[['ds']]]))
figure = ph_a.plot(forecast_a)
figure.show()

In [ ]:
ph_b = Prophet(holidays=holidays, holidays_prior_scale=10.0)
ph_b.fit(df_train_b)
forecast_b = ph_b.predict(pd.concat([df_train_b[['ds']],df_test_b[['ds']]]))
figure = ph_b.plot(forecast_b)
figure.show()

In [ ]:
forecast_a = ph_a.predict(df_test_a[['ds']])
forecast_a["yhat"] = forecast_a["yhat"] * std_a + mean_a
forecast_b = ph_b.predict(df_test_b[['ds']])
forecast_b["yhat"] = forecast_b["yhat"] * std_b + mean_b
forecast = forecast_a[["ds","yhat"]].merge(forecast_b[["ds","yhat"]],on="ds")
forecast["ds"] = forecast["ds"].apply(lambda ds:str(ds).replace("-","/"))

submission = forecast.rename(columns={"ds":"日期","yhat_x":"A厂","yhat_y":"B厂"})
submission["日期"] = submission["日期"].apply(lambda ds:str(ds).split(" ")[0])
submission

In [ ]:
submission.to_csv("submit-prophet.csv",index=False)

The end.